In [1]:
from fastai.imports import *
from fastai.structured import *

In [2]:
!ls

 Data_Train.xlsx    'lgbm model.ipynb'	      test.csv	      train.csv
 edit.ipynb	     sample.csv		      test-raw	      trainRaw
 fastai		     Sample_submission.xlsx   testRaw	      tsp.csv
 featherwali.ipynb   sub.csv		      Test_set.xlsx   tst.csv
 final.csv	     submission.xlsx	      ticket.ipynb    Untitled.ipynb


In [3]:
df_raw = pd.read_feather('trainRaw')

In [4]:
df_raw.head(2)

,Airline,Source,Destination,Route,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,timeYear,...,timeIs_month_end,timeIs_month_start,timeIs_quarter_end,timeIs_quarter_start,timeIs_year_end,timeIs_year_start,timeHour,timeMinute,timeSecond,timeElapsed
0,Air India,Banglore,New Delhi,BLR → BOM → AMD → DEL,23:55 02 Mar,39h 5m,2 stops,No info,17135,2019,...,False,False,False,False,False,False,8,50,0,1546505400
1,Vistara,Banglore,New Delhi,BLR → DEL,09:40,2h 40m,non-stop,No info,21520,2019,...,False,False,False,False,False,False,7,0,0,1546498800


In [5]:
df_trn, y_trn, nas = proc_df(df_raw, 'Price')

In [6]:
df_trn.head()

,Airline,Source,Destination,Route,Arrival_Time,Duration,Total_Stops,Additional_Info,timeYear,timeMonth,...,timeIs_month_end,timeIs_month_start,timeIs_quarter_end,timeIs_quarter_start,timeIs_year_end,timeIs_year_start,timeHour,timeMinute,timeSecond,timeElapsed
0,2,1,6,4,1343,280,2,9,2019,1,...,False,False,False,False,False,False,8,50,0,1546505400
1,11,1,6,19,702,239,5,9,2019,1,...,False,False,False,False,False,False,7,0,0,1546498800
2,5,1,6,6,987,335,1,9,2019,1,...,False,False,False,False,False,False,8,55,0,1546505700
3,2,1,6,6,859,313,1,9,2019,1,...,False,False,False,False,False,False,6,45,0,1546497900
4,4,1,6,19,228,241,5,9,2019,1,...,False,False,False,False,False,False,22,20,0,1546554000


In [7]:
y_trn[:10]

array([17135, 21520, 26890, 23677, 11934, 17461, 14305, 31825, 10067, 26890])

In [8]:
def split_vals(a,n): return a[:n].copy(), a[n:].copy()

n_valid = 2000  # same as Kaggle's test set size
n_trn = len(df_raw)-n_valid
raw_train, raw_valid = split_vals(df_raw, n_trn)
X_train, X_valid = split_vals(df_trn, n_trn)
y_train, y_valid = split_vals(y_trn, n_trn)

X_train.shape, y_train.shape, X_valid.shape, y_valid.shape

((8683, 24), (8683,), (2000, 24), (2000,))

In [9]:
import lightgbm as lgb

In [10]:
d_train = lgb.Dataset(X_train, label= y_train)
d_test=lgb.Dataset(X_valid, label=y_valid, reference=d_train,)

In [11]:


params = {'num_leaves': 25,
         'min_data_in_leaf': 30, 
         'objective':'regression',
         'max_depth': 20,
         'learning_rate': 0.001,
         "boosting": "gbdt",
         "metric": 'rmse',
         "verbosity": -1}



In [12]:
m = lgb.train(params,
                       d_train,
                       valid_sets=[d_train, d_test],
                       num_boost_round=5000,
                       early_stopping_rounds=100)



[1]	training's rmse: 4733.2	valid_1's rmse: 4025.53
Training until validation scores don't improve for 100 rounds.
[2]	training's rmse: 4729.62	valid_1's rmse: 4022.59
[3]	training's rmse: 4726.05	valid_1's rmse: 4019.66
[4]	training's rmse: 4722.49	valid_1's rmse: 4016.73
[5]	training's rmse: 4718.93	valid_1's rmse: 4013.8
[6]	training's rmse: 4715.37	valid_1's rmse: 4010.88
[7]	training's rmse: 4711.81	valid_1's rmse: 4007.95
[8]	training's rmse: 4708.26	valid_1's rmse: 4005.02
[9]	training's rmse: 4704.7	valid_1's rmse: 4002.09
[10]	training's rmse: 4701.16	valid_1's rmse: 3999.17
[11]	training's rmse: 4697.62	valid_1's rmse: 3996.26
[12]	training's rmse: 4694.08	valid_1's rmse: 3993.34
[13]	training's rmse: 4690.55	valid_1's rmse: 3990.43
[14]	training's rmse: 4687.02	valid_1's rmse: 3987.53
[15]	training's rmse: 4683.49	valid_1's rmse: 3984.62
[16]	training's rmse: 4679.97	valid_1's rmse: 3981.72
[17]	training's rmse: 4676.46	valid_1's rmse: 3978.83
[18]	training's rmse: 4672.95	v

[210]	training's rmse: 4073.24	valid_1's rmse: 3467.84
[211]	training's rmse: 4070.48	valid_1's rmse: 3465.51
[212]	training's rmse: 4067.73	valid_1's rmse: 3463.29
[213]	training's rmse: 4064.98	valid_1's rmse: 3460.98
[214]	training's rmse: 4062.23	valid_1's rmse: 3458.63
[215]	training's rmse: 4059.49	valid_1's rmse: 3456.32
[216]	training's rmse: 4056.75	valid_1's rmse: 3454.01
[217]	training's rmse: 4054.01	valid_1's rmse: 3451.71
[218]	training's rmse: 4051.29	valid_1's rmse: 3449.56
[219]	training's rmse: 4048.56	valid_1's rmse: 3447.22
[220]	training's rmse: 4045.84	valid_1's rmse: 3445.12
[221]	training's rmse: 4043.12	valid_1's rmse: 3442.83
[222]	training's rmse: 4040.41	valid_1's rmse: 3440.69
[223]	training's rmse: 4037.69	valid_1's rmse: 3438.42
[224]	training's rmse: 4034.98	valid_1's rmse: 3436.15
[225]	training's rmse: 4032.28	valid_1's rmse: 3434.01
[226]	training's rmse: 4029.58	valid_1's rmse: 3431.75
[227]	training's rmse: 4026.88	valid_1's rmse: 3429.62
[228]	trai

[404]	training's rmse: 3601.02	valid_1's rmse: 3082.8
[405]	training's rmse: 3598.89	valid_1's rmse: 3081.17
[406]	training's rmse: 3596.77	valid_1's rmse: 3079.4
[407]	training's rmse: 3594.64	valid_1's rmse: 3077.73
[408]	training's rmse: 3592.51	valid_1's rmse: 3075.94
[409]	training's rmse: 3590.4	valid_1's rmse: 3074.33
[410]	training's rmse: 3588.28	valid_1's rmse: 3072.53
[411]	training's rmse: 3586.15	valid_1's rmse: 3070.74
[412]	training's rmse: 3584.05	valid_1's rmse: 3069.08
[413]	training's rmse: 3581.94	valid_1's rmse: 3067.32
[414]	training's rmse: 3579.83	valid_1's rmse: 3065.65
[415]	training's rmse: 3577.73	valid_1's rmse: 3063.89
[416]	training's rmse: 3575.62	valid_1's rmse: 3062.21
[417]	training's rmse: 3573.53	valid_1's rmse: 3060.47
[418]	training's rmse: 3571.44	valid_1's rmse: 3058.88
[419]	training's rmse: 3569.34	valid_1's rmse: 3057.11
[420]	training's rmse: 3567.25	valid_1's rmse: 3055.52
[421]	training's rmse: 3565.17	valid_1's rmse: 3053.74
[422]	trainin

[608]	training's rmse: 3217.26	valid_1's rmse: 2787.2
[609]	training's rmse: 3215.61	valid_1's rmse: 2785.99
[610]	training's rmse: 3213.97	valid_1's rmse: 2784.74
[611]	training's rmse: 3212.34	valid_1's rmse: 2783.56
[612]	training's rmse: 3210.69	valid_1's rmse: 2782.53
[613]	training's rmse: 3209.06	valid_1's rmse: 2781.34
[614]	training's rmse: 3207.43	valid_1's rmse: 2780.17
[615]	training's rmse: 3205.82	valid_1's rmse: 2778.75
[616]	training's rmse: 3204.18	valid_1's rmse: 2777.67
[617]	training's rmse: 3202.56	valid_1's rmse: 2776.5
[618]	training's rmse: 3200.94	valid_1's rmse: 2775.3
[619]	training's rmse: 3199.32	valid_1's rmse: 2774.11
[620]	training's rmse: 3197.71	valid_1's rmse: 2772.72
[621]	training's rmse: 3196.09	valid_1's rmse: 2771.7
[622]	training's rmse: 3194.48	valid_1's rmse: 2770.5
[623]	training's rmse: 3192.87	valid_1's rmse: 2769.31
[624]	training's rmse: 3191.26	valid_1's rmse: 2768.24
[625]	training's rmse: 3189.66	valid_1's rmse: 2767.08
[626]	training'

[822]	training's rmse: 2913.45	valid_1's rmse: 2561.78
[823]	training's rmse: 2912.24	valid_1's rmse: 2560.92
[824]	training's rmse: 2911.02	valid_1's rmse: 2560.07
[825]	training's rmse: 2909.79	valid_1's rmse: 2559.2
[826]	training's rmse: 2908.59	valid_1's rmse: 2558.15
[827]	training's rmse: 2907.37	valid_1's rmse: 2557.35
[828]	training's rmse: 2906.17	valid_1's rmse: 2556.29
[829]	training's rmse: 2904.96	valid_1's rmse: 2555.38
[830]	training's rmse: 2903.75	valid_1's rmse: 2554.53
[831]	training's rmse: 2902.56	valid_1's rmse: 2553.49
[832]	training's rmse: 2901.35	valid_1's rmse: 2552.65
[833]	training's rmse: 2900.16	valid_1's rmse: 2551.67
[834]	training's rmse: 2898.95	valid_1's rmse: 2550.83
[835]	training's rmse: 2897.76	valid_1's rmse: 2549.93
[836]	training's rmse: 2896.56	valid_1's rmse: 2549.09
[837]	training's rmse: 2895.37	valid_1's rmse: 2548.11
[838]	training's rmse: 2894.18	valid_1's rmse: 2547.26
[839]	training's rmse: 2892.99	valid_1's rmse: 2546.23
[840]	train

...

[3961]	training's rmse: 1794.84	valid_1's rmse: 1677.37
[3962]	training's rmse: 1794.74	valid_1's rmse: 1677.27
[3963]	training's rmse: 1794.64	valid_1's rmse: 1677.2
[3964]	training's rmse: 1794.53	valid_1's rmse: 1677.13
[3965]	training's rmse: 1794.44	valid_1's rmse: 1677.08
[3966]	training's rmse: 1794.33	valid_1's rmse: 1676.99
[3967]	training's rmse: 1794.22	valid_1's rmse: 1677.01
[3968]	training's rmse: 1794.12	valid_1's rmse: 1676.95
[3969]	training's rmse: 1794.02	valid_1's rmse: 1676.85
[3970]	training's rmse: 1793.93	valid_1's rmse: 1676.77
[3971]	training's rmse: 1793.83	valid_1's rmse: 1676.68
[3972]	training's rmse: 1793.72	valid_1's rmse: 1676.7
[3973]	training's rmse: 1793.61	valid_1's rmse: 1676.61
[3974]	training's rmse: 1793.49	valid_1's rmse: 1676.54
[3975]	training's rmse: 1793.38	valid_1's rmse: 1676.48
[3976]	training's rmse: 1793.28	valid_1's rmse: 1676.41
[3977]	training's rmse: 1793.19	valid_1's rmse: 1676.31
[3978]	training's rmse: 1793.09	valid_1's rmse: 16

[4142]	training's rmse: 1776.37	valid_1's rmse: 1666.56
[4143]	training's rmse: 1776.27	valid_1's rmse: 1666.49
[4144]	training's rmse: 1776.14	valid_1's rmse: 1666.42
[4145]	training's rmse: 1776.06	valid_1's rmse: 1666.39
[4146]	training's rmse: 1775.96	valid_1's rmse: 1666.38
[4147]	training's rmse: 1775.83	valid_1's rmse: 1666.31
[4148]	training's rmse: 1775.75	valid_1's rmse: 1666.21
[4149]	training's rmse: 1775.63	valid_1's rmse: 1666.14
[4150]	training's rmse: 1775.53	valid_1's rmse: 1666.07
[4151]	training's rmse: 1775.4	valid_1's rmse: 1666
[4152]	training's rmse: 1775.32	valid_1's rmse: 1665.91
[4153]	training's rmse: 1775.25	valid_1's rmse: 1665.89
[4154]	training's rmse: 1775.15	valid_1's rmse: 1665.82
[4155]	training's rmse: 1775.03	valid_1's rmse: 1665.75
[4156]	training's rmse: 1774.95	valid_1's rmse: 1665.72
[4157]	training's rmse: 1774.84	valid_1's rmse: 1665.71
[4158]	training's rmse: 1774.74	valid_1's rmse: 1665.65
[4159]	training's rmse: 1774.61	valid_1's rmse: 1665

[4358]	training's rmse: 1756.04	valid_1's rmse: 1657.49
[4359]	training's rmse: 1755.91	valid_1's rmse: 1657.45
[4360]	training's rmse: 1755.82	valid_1's rmse: 1657.41
[4361]	training's rmse: 1755.74	valid_1's rmse: 1657.42
[4362]	training's rmse: 1755.66	valid_1's rmse: 1657.35
[4363]	training's rmse: 1755.54	valid_1's rmse: 1657.3
[4364]	training's rmse: 1755.45	valid_1's rmse: 1657.27
[4365]	training's rmse: 1755.39	valid_1's rmse: 1657.28
[4366]	training's rmse: 1755.31	valid_1's rmse: 1657.2
[4367]	training's rmse: 1755.23	valid_1's rmse: 1657.22
[4368]	training's rmse: 1755.14	valid_1's rmse: 1657.15
[4369]	training's rmse: 1755.02	valid_1's rmse: 1657.11
[4370]	training's rmse: 1754.93	valid_1's rmse: 1657.08
[4371]	training's rmse: 1754.87	valid_1's rmse: 1657.09
[4372]	training's rmse: 1754.78	valid_1's rmse: 1657.04
[4373]	training's rmse: 1754.69	valid_1's rmse: 1657
[4374]	training's rmse: 1754.61	valid_1's rmse: 1657.01
[4375]	training's rmse: 1754.53	valid_1's rmse: 1656.

[4618]	training's rmse: 1733.91	valid_1's rmse: 1647.84
[4619]	training's rmse: 1733.84	valid_1's rmse: 1647.85
[4620]	training's rmse: 1733.76	valid_1's rmse: 1647.8
[4621]	training's rmse: 1733.66	valid_1's rmse: 1647.72
[4622]	training's rmse: 1733.6	valid_1's rmse: 1647.74
[4623]	training's rmse: 1733.53	valid_1's rmse: 1647.71
[4624]	training's rmse: 1733.46	valid_1's rmse: 1647.66
[4625]	training's rmse: 1733.38	valid_1's rmse: 1647.6
[4626]	training's rmse: 1733.31	valid_1's rmse: 1647.57
[4627]	training's rmse: 1733.24	valid_1's rmse: 1647.52
[4628]	training's rmse: 1733.17	valid_1's rmse: 1647.48
[4629]	training's rmse: 1733.09	valid_1's rmse: 1647.5
[4630]	training's rmse: 1732.99	valid_1's rmse: 1647.42
[4631]	training's rmse: 1732.92	valid_1's rmse: 1647.4
[4632]	training's rmse: 1732.86	valid_1's rmse: 1647.41
[4633]	training's rmse: 1732.78	valid_1's rmse: 1647.36
[4634]	training's rmse: 1732.68	valid_1's rmse: 1647.28
[4635]	training's rmse: 1732.61	valid_1's rmse: 1647.

[4848]	training's rmse: 1717.21	valid_1's rmse: 1640.37
[4849]	training's rmse: 1717.15	valid_1's rmse: 1640.35
[4850]	training's rmse: 1717.07	valid_1's rmse: 1640.35
[4851]	training's rmse: 1717	valid_1's rmse: 1640.3
[4852]	training's rmse: 1716.93	valid_1's rmse: 1640.27
[4853]	training's rmse: 1716.86	valid_1's rmse: 1640.22
[4854]	training's rmse: 1716.79	valid_1's rmse: 1640.21
[4855]	training's rmse: 1716.72	valid_1's rmse: 1640.23
[4856]	training's rmse: 1716.64	valid_1's rmse: 1640.2
[4857]	training's rmse: 1716.58	valid_1's rmse: 1640.17
[4858]	training's rmse: 1716.5	valid_1's rmse: 1640.18
[4859]	training's rmse: 1716.43	valid_1's rmse: 1640.15
[4860]	training's rmse: 1716.35	valid_1's rmse: 1640.16
[4861]	training's rmse: 1716.28	valid_1's rmse: 1640.11
[4862]	training's rmse: 1716.2	valid_1's rmse: 1640.12
[4863]	training's rmse: 1716.13	valid_1's rmse: 1640.06
[4864]	training's rmse: 1716.07	valid_1's rmse: 1640.03
[4865]	training's rmse: 1715.99	valid_1's rmse: 1640.04

In [13]:
y_pred = m.predict(X_valid)

In [14]:
y_pred

array([12274.44351, 13679.07489,  9755.46688, ...,  5232.71153, 10932.48621, 10927.20997])

In [75]:
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_pred,y_valid)

ValueError: Classification metrics can't handle a mix of continuous and multiclass targets

In [67]:
sub=pd.DataFrame(y_pred)

In [68]:
sub=sub.astype(int)

In [69]:
sub.head()

,0
0,12274
1,13679
2,9755
3,9753
4,4789


In [70]:
y_valid

array([17049, 14388, 10844, ...,  4668,  7408, 11185])

In [72]:
def rmse(x,y): return math.sqrt(((x-y)**2).mean())

def print_score(m):
    res = [rmse(m.predict(X_train), y_train), rmse(m.predict(X_valid), y_valid),
                m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

In [74]:
m.score(X_train, y_train)

AttributeError: 'Booster' object has no attribute 'score'